Obtain NYT URL's

In [66]:
#Import necessary libraries
import time
import json
import requests
import urllib
import lxml.html
from bs4 import BeautifulSoup
import newspaper
import trafilatura
import calendar
import os
from urllib.parse import urlparse
import datetime
import pytz

In [67]:
nyt_api_key = "AUtPPNuICzon3X9uEaX2k4Dgs0YGBA25"
START_DATE = datetime.datetime(2015,1,1,0,0, tzinfo=pytz.timezone("utc"))
END_DATE = datetime.datetime.now(pytz.timezone("utc"))

In [68]:
"Standard values"
POLITICS_SECTION = "politics"
QUERY_LIST = ["Donald Trump", "Joe Biden"]
FILTER_LIST = ["headline", "lead_paragraph"]
nyt_api_key = "AUtPPNuICzon3X9uEaX2k4Dgs0YGBA25" ##Move to config file

In [69]:
def create_url(tags=None,news_desk=POLITICS_SECTION, section = "Washington", filters=FILTER_LIST, begin_date=START_DATE,
 end_date=END_DATE, page="0"):
    """
    Create request url for API based on query search parameters passed to the
        function.
    
    Inputs:
        tags (lst): list of tags (strings) to look for. The tags to filter for
            are looked in the filters defined in the "filters" parameter.
        filters (lst): list of filters where to look tags. They can be "headline",
            "lead_paragraph" and/or "body"
        begin_date (str): 8 digits (YYYYMMDD) string that specify the begin date
            or from when to start looking for articles.
        end_date (str): 8 digits (YYYYMMDD) string that specify the end date or
            until when to stop looking for articles.
        page (str): number of page string that states where to look for articles.

    Return (str): URL string with query to send request to NYT Article Search 
        API
    """
    begin_date = begin_date.strftime("%Y%m%d")
    end_date = end_date.strftime("%Y%m%d")

    endpoint = "https://api.nytimes.com/svc/search/v2/articlesearch.json?" 
    

    #headline = "(headline=" + " OR ".join(filters_copy) + ")"
    #news_desk = "(news_desk)" + news_desk


    if tags:
        tags_copy = tags[:]
        filters_copy = filters[:]
        for i,tag in enumerate(tags_copy):
            tags_copy[i] = "\"" + tag + "\""
        for i, fil in enumerate(filters_copy):
            filters_copy[i] = fil + ":(" + " OR ".join(tags_copy) + ")"
        fq = "fq=" + " OR ".join(filters_copy) + " AND news_desk=" + news_desk + " AND section=" + section
    else:
        fq = "fq=" + "news_desk=" + news_desk + "AND section=" + section
    url = endpoint + fq + "&begin_date=" + begin_date + "&end_date=" + end_date +\
            "&page=" + page + "&sort=oldest" + "&api-key=" + nyt_api_key

    return url

create_url()


'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk=politicsAND section=Washington&begin_date=20150101&end_date=20230731&page=0&sort=oldest&api-key=AUtPPNuICzon3X9uEaX2k4Dgs0YGBA25'

In [70]:
def make_request(tags=None,news_desk=POLITICS_SECTION, section = "Washington", filters=FILTER_LIST, begin_date=START_DATE,
 end_date=END_DATE, page="0"):
    """
    Make a GET request to the NYT Article Search API with a request delay of 6
        seconds to avoid reaching request limit of 60 requests per minute.

    Inputs:
        tags (lst): list of tags (strings) to look for. The tags to filter for
            are looked in the body, headline and byline of the articles.
        filters (lst): list of filters where to look tags. They can be "headline",
            "lead_paragraph" and/or "body"
        begin_date (str): 8 digits (YYYYMMDD) string that specify the begin date
            or from when to start looking for articles.
        end_date (str): 8 digits (YYYYMMDD) string that specify the end date or
            until when to stop looking for articles.
        page (str): number of page string that states where to look for articles.
    
    Return (Response): API request response with specified query parameters
    """

    url = create_url(tags, news_desk, section, filters, begin_date, end_date, page)
    time.sleep(1)
    resp = requests.get(url)
    resp_json = json.loads(resp.text)

    return resp_json



In [71]:
resp_json = make_request()
resp_json["response"]["meta"]


{'hits': 129724, 'offset': 0, 'time': 53}

In [59]:

def make_link_absolute(rel_url, current_url):
    """
    Given a relative URL like "/abc/def" or "?page=2"
    and a complete URL like "https://example.com/1/2/3" this function will
    combine the two yielding a URL like "https://example.com/abc/def"

    Parameters:
        * rel_url:      a URL or fragment
        * current_url:  a complete URL used to make the request that contained a link to rel_url

    Returns:
        A full URL with protocol & domain that refers to rel_url.
    """
    url = urlparse(current_url)
    if rel_url.startswith("/"):
        return f"{url.scheme}://{url.netloc}{rel_url}"
    elif rel_url.startswith("?"):
        return f"{url.scheme}://{url.netloc}{url.path}{rel_url}"
    else:
        return rel_url

def obtain_articles_info(json_resp):
    articles = json_resp["response"]["docs"]
    for article in articles:
        multimedia = article["multimedia"]
        for item in multimedia:
            url = "/" + item["url"]
            item["url"] = make_link_absolute(url,"https://www.nytimes.com/images/")
    return articles

def crawl(tags=None,news_desk=POLITICS_SECTION, section = "Washington", 
        filters=FILTER_LIST, begin_date="20210101", end_date="20230101"):
        """
        Crawl NYT to obtain all the information
        """

        page_responses = []
        
        resp = make_request()
        articles = obtain_articles_info(resp)
        page_responses.append(articles)
        # Get number of articles that match our query search parameters
        hits = resp_json["response"]["meta"]["hits"]

        # Get maximum number of pages we can query
        max_pages = int(hits / 10)

        for page_n in range(1, max_pages + 1):
            #NYT has a limit of 5 requests per minute
            time.sleep(12)
            resp = make_request(page = str(page_n))
            status = resp.get("status")
            if status== "OK":
                articles = obtain_articles_info(resp)
                page_responses.append(articles)
            else:
                print(resp)
                break
        return page_responses

test = crawl(begin_date="20230101", end_date="20230131")

KeyboardInterrupt: 